In [11]:
# This script is the process of determining the genes for analysis in MutSigCVsyn
# 1. Gencode v19 KNOWN protein coding genes
# 2. PCAWG protein coding genes(In MAF file) - include both protein coding and not protein coding
# 2.1 if the transcript information we grab from Gencode v19 matches mutations in MAF files
        # Directly read intermediate file in 2.1.1

# Author: Yiyun

In [1]:
import pandas as pd
import os, pickle, mmap
from multiprocessing import Pool
from tqdm import tqdm

In [3]:
### Directories
dir_forgene = '../data/intermediate/for_gene' ### This is the directory especially for this script

dir_validate = os.path.join(dir_forgene,'validate_cov_maf') ###validate folder contain the intermediate file of SNP validation result
dir_maf_split = os.path.join(dir_forgene, 'maf_split/') ### Splitted maf for parallel running
dir_ref = '../data/proc_refs/'

***1. Protein coding genes collected in Genscript***

In [5]:
### Read processed annotation file and name file
dict_transcript_info = pickle.load(open(os.path.join(dir_ref,'dict_transcript_info_062121.pkl'),'rb'))
dict_name = pickle.load(open(os.path.join(dir_ref,'dict_name_062121.pkl'),'rb'))

In [6]:
### Get list of genes in transcript info(because it only keeps the principle/longest transcript for each gene)
lgencode = []
for transcript in dict_transcript_info:
    gene_name = dict_name[transcript]
    lgencode.append(gene_name)

In [7]:
print(f'Number of all protein coding gene in Gencode annotation file is {len(lgencode)}')

Number of all protein coding gene in Gencode annotation file is 19621


***2. All genes in MAF file -- This is the big maf file that contains all mutations***   
--Have intermediate file, don't need to run  
    2.1 Check if all SNPs are in the coding/non-coding(intron/UTR) region as in the transcript info  

In [8]:
### The folder for splitted maf file
coding_mut = ['Missense_Mutation','Nonsense_Mutation','Silent','Nonstop_Mutation','Start_Codon_SNP']
noncoding_mut = ['Intron',"3'UTR","5'UTR"]

In [9]:
### Get the gene name list in the maf file
def get_geneinmaf(maff):
    global dir_maf_split,coding_mut,noncoding_mut,dir_validate
    
    print(f'START {maff}')
    ### If the file had already been mapped, skip the file
    if maff+'.pkl' in os.listdir(dir_validate):
        print(f'{maff} already finished')
        return

    merged_maf = os.path.join(dir_maf_split,maff)
    # Read maf
    df_maf = pd.read_csv(merged_maf,sep = '\t', header = None)
    
    # Get all gene list and filter out the genes not in gencode
    lgene = df_maf.iloc[:,0].unique().tolist() # Get genelist
    lgene = [i for i in lgene if i in lgencode]
    lgene_notpass = [] # set a list for the not qualified genes
    
    # set genes, mutation type(syn, nysn...), mutation categ(SNP, DNP) as index , only keep SNPs
    df_maf = df_maf.set_index([0,5,6], drop = True)
    df_maf = df_maf.iloc[df_maf.index.get_level_values(6) == 'SNP'] ### Limit the maf file to SNPs
    
    # iterate through genes to check every mutations in that gene match coverage information(transcript ino)
    for genes in lgene:
        quality_flag = 1 # Set up initial quality flag for gene, assume it's maf and coverage SNP matches
        transcript = 0# set up initial transcript to avoid using the last one
        
        # Find transcript_id of that gene
        ltranscript = []
        for k,v in dict_name.items():
            if v == genes:
                ltranscript.append(k)
        if not ltranscript: ### If the transcript is not found for the gene, flag the gene and continue the loop
            print(f'no transcript found for the {genes}');quality_flag = 0
            continue
        if len(ltranscript) == 1: transcript = ltranscript[0]
        else: 
            # print(f'{genes} has multiple transcript{ltranscript}')
            transcript = [i for i in ltranscript if i in dict_transcript_info.keys()]
            if len(transcript) == 1: transcript = transcript[0]
        
        # 1. Find the CDS information of that transcript
        CDS_seq_info = dict_transcript_info[transcript]['CDS']
        UTR_seq_info = dict_transcript_info[transcript]['UTR']
        transcript_seq_info = dict_transcript_info[transcript]['transcript']
        
        # 2. Get the gene-level dataframe with mutations - for coding and non-coding
        df_maf_gene = df_maf.iloc[df_maf.index.get_level_values(0) == genes]
        df_maf_gene_ncd = df_maf_gene.iloc[df_maf_gene.index.get_level_values(5).isin(noncoding_mut)]
        df_maf_gene_cd = df_maf_gene.iloc[df_maf_gene.index.get_level_values(5).isin(coding_mut)]
        ncd_mut_pos = df_maf_gene_ncd[2].unique().tolist()
        cd_mut_pos = df_maf_gene_cd[2].unique().tolist()
        
        # 3. Check if the coding/non-coding position in CDS sequence
        if cd_mut_pos:
            for value in cd_mut_pos:
                value = int(value)
                if not list(filter(lambda x: value<x[1] and value>x[0], CDS_seq_info)):quality_flag = 0  #if mutation doesn't fall in CDS
        if ncd_mut_pos:
            for value in ncd_mut_pos:  # it should be in exon, not in CDS, maybe in UTR
                value = int(value)
                if list(filter(lambda x: value<x[1] and value>x[0], CDS_seq_info))\
                or not ((value< transcript_seq_info[1]) and (value > transcript_seq_info[0])): quality_flag = 0
        # if the quality_flag = 0, means that not all mutations matches the transcript info/gene not found, we exclude the gene
        if quality_flag == 0:
            lgene_notpass.append(genes)
    
    lgene_final = [i for i in lgene if i not in lgene_notpass]
    print(len(lgene));print(len(lgene_final));print(len(lgene_notpass))
    pickle.dump(lgene_final, open(os.path.join(dir_validate,maff+'.pkl'),'wb'))

In [10]:
### Running parallel for only one time
p = Pool(2)
res = p.map(get_geneinmaf, os.listdir(dir_maf_split))
p.close()
p.join()

START split.maf.bsSTART split.maf.ae

split.maf.bs already finishedsplit.maf.ae already finished

START split.maf.bvSTART split.maf.av

split.maf.bv already finishedsplit.maf.av already finished

START split.maf.ba
split.maf.ba already finished
START split.maf.ax
split.maf.ax already finishedSTART split.maf.az

START split.maf.aksplit.maf.az already finished
split.maf.ak already finished
START split.maf.an
split.maf.an already finished
START split.maf.au

START split.maf.busplit.maf.au already finished

START split.maf.bzsplit.maf.bu already finished

split.maf.bz already finishedSTART split.maf.bn

START split.maf.btsplit.maf.bn already finished

split.maf.bt already finishedSTART split.maf.al
START split.maf.bj

split.maf.al already finishedsplit.maf.bj already finished

START split.maf.aiSTART split.maf.bq

split.maf.bq already finishedsplit.maf.ai already finished

START split.maf.bfSTART split.maf.bi

split.maf.bf already finishedsplit.maf.bi already finished

START split.maf.bdST

**2.1.1 Anlyze the mapped gene and all intersections**

In [12]:
list_pass_genes_all = []
for pklPasslists in os.listdir(dir_validate):
    list_pass_genes = pickle.load(open(os.path.join(dir_validate,pklPasslists),'rb'))
    list_pass_genes_all.extend(list_pass_genes)

In [13]:
len(set(list_pass_genes_all))

18638

In [14]:
print(f'Number of intersection of Gencode protein coding genes and MAF genes is \
{len(set(lgencode).intersection(set(list_pass_genes_all)))}')

Number of intersection of Gencode protein coding genes and MAF genes is 18638


In [36]:
### Save intersected genelist
lgene = list(set(lgencode).intersection(set(list_pass_genes_all)))
# pickle.dump(lgene, open(os.path.join(dir_ref,'gene_name_list_102121.pkl'),'wb'))
# with open(os.path.join(dir_ref,'gene_name_list_102121.txt'),'w') as f:
#     for gene in lgene:
#         f.write(gene+'\n')

In [37]:
### Create transcript- name dict for intersection genes
dict_name_transcript = {}
for k,v in dict_name.items():
    if v in lgene:
        if v not in dict_name_transcript:
            dict_name_transcript[v] = []
        dict_name_transcript[v].append(k)
dict_gene_forcov = {}
for k,v in dict_name_transcript.items():
    if k in lgene:
        transcript_id = [i for i in v if i in dict_transcript_info.keys()][0]
        dict_gene_forcov[transcript_id] = k
# pickle.dump(dict_gene_forcov, open(os.path.join(dir_ref,'dict_name_forcov_102121.pkl'),'wb'))